## Homework 05: First Order ODEs
### PHYS420 — Intro to Computational Physics — Fall 2025  
### Hayden Dauphin

<img src="./Images/hw5q1.png" width="700"/>

In [21]:
# --- Dependencies --- 
import matplotlib.pyplot as plt
import numpy as np
import scipy as sci 
from scipy.integrate import solve_ivp

In [19]:
# ---- Problem 1: Coding and testing ----

# --- Code 1: Explicit Euler --- 

def explicit_euler(f, timeInterval, initCond, stepSize):
    ''' 
    Function to compute first order ODEs with the explicit Euler method
    Arguments: 
        f: function 
        timeInterval: interval of time to be evaluated over
        initCond: initial condition at start of timeInterval 
        stepSize: size of each step t_{i+1}
    Returns: 
        t_vals: array of time values 
        y_vals: array of solution values at each time value
    '''

    t_start, t_stop = timeInterval  
    num_steps = int((t_stop - t_start) / stepSize)  # calculate number of steps by dividing length of interval by size of steps 
    t_vals = np.linspace(t_start, t_stop, num_steps + 1)    # create array of time values 
    y_vals = np.zeros_like(t_vals, dtype=float)  # create array of zeros with length = len(t_vals) and data type = type(t_vals)
    y_vals[0] = initCond    # set initial condition as first y value 

    # -- Loop to calculate solutions at each time point with equation for explicit Euler: y_(i+1) = y_i + h * f(t_i, y_i)
    for i in range(num_steps):
        y_vals[i+1] = y_vals[i] + stepSize * f(t_vals[i], y_vals[i])
    
    return t_vals, y_vals

# -- Test function for explicit Euler --
print("Explicit Euler test: ")
f1 = lambda t, y: -y 
times1, sols1 = explicit_euler(f1, (0,5), 1.0, 0.001)
print("Final y = ", sols1[-1])
print("Exact = ", np.exp(-5))
percenterr = (np.abs((np.exp(-5) - sols1[-1])/np.exp(-5))*100)
print("Percent error: ", percenterr, "%")


Explicit Euler test: 
Final y =  0.006721111959865623
Exact =  0.006737946999085467
Percent error:  0.24985413542328475 %


In [20]:
# --- Code 2: RKF45 --- 

# - Fehlberg 4(5) coefficients - 
c = [0, 1/4, 3/8, 12/13, 1, 1/2]
a = [
    [0,0,0,0,0],
    [1/4,0,0,0,0],
    [3/32,9/32,0,0,0],
    [1932/2197,-7200/2197,7296/2197,0,0],
    [439/216,-8,3680/513,-845/4104,0],
    [-8/27,2,-3544/2565,1859/4104,-11/40]
]
# - 4th and 5th order weights -
b5 = [16/135,0,6656/12825,28561/56430,-9/50,2/55]
b4 = [25/216,0,1408/2565,2197/4104,-1/5,0]

def rkf45(f, t0, tf, y0, relTol=1e-8, absTol=1e-10, h=0.01): 
    ''' 
    RKF45 method of solving first order ODEs
    Arguments: 
        f: function 
        t: time interval 
        y0: initial condition 
        relTol: relative tolerance 
        absTol: absolute tolerance 
        h: initial step size 
    Returns: 
        ts: array of the time values solution computed for 
        ys: array of solutions 
        errs: array of error estimates for each accepted step 
        n_accept: number of accepted 
        n_reject: number of rejected steps 
    '''
    t = t0 
    y = np.array(y0, dtype=float)
    ts = [t]
    ys = [y.copy()]
    errs = [0.0]
    n_accept = n_reject = 0 


    # failsafe counter
    max_steps = 100000
    count = 0

    while t < tf and count < max_steps: 
        if t + h > tf: 
            h = tf - t 
        
        #compute k 1-6 
        k1 = f(t, y)
        k2 = f(t + c[1]*h, y + h*a[1][0]*k1)
        k3 = f(t + c[2]*h, y + h*(a[2][0]*k1 + a[2][1]*k2))
        k4 = f(t + c[3]*h, y + h*(a[3][0]*k1 + a[3][1]*k2 + a[3][2]*k3))
        k5 = f(t + c[4]*h, y + h*(a[4][0]*k1 + a[4][1]*k2 + a[4][2]*k3 + a[4][3]*k4))
        k6 = f(t + c[5]*h, y + h*(a[5][0]*k1 + a[5][1]*k2 + a[5][2]*k3 + a[5][3]*k4 + a[5][4]*k5))

        Ks = np.array([k1, k2, k3, k4, k5, k6])
        y5 = y + h * np.dot(b5, Ks)
        y4 = y + h * np.dot(b4, Ks)
        err = np.linalg.norm(y5 - y4)

        # scaled error test
        scale = absTol + relTol * np.maximum(np.abs(y), np.abs(y5))
        err_norm = err/np.linalg.norm(scale)

        count += 1

        if err_norm <= 1.0: 
            t += h
            y = y5 
            ts.append(t)
            ys.append(y.copy())
            errs.append(err_norm)
            n_accept += 1
        else: 
            n_reject += 1 

        # update step size 
        if err_norm == 0: 
            scaling = 2.0 
        else: 
            scaling = 0.9 * err_norm**(-0.2) 
        scaling = np.clip(scaling, 0.2, 5.0)
        h *= scaling 
    if count >= max_steps:
        print("Warning: maximum number of steps reached. Integration might be unfinished.")
    return np.array(ts), np.array(ys), np.array(errs), n_accept, n_reject

print("\nRKF45 test:")

f2 = lambda t2, y2: y2 *(1 - y2)
times2, sols2, errEst, acc, rej = rkf45(f2, 0, 10, 0.1, h=0.001)
print(f"Final y = {sols2[-1]:.6f}, expected = ~1.0, accepted = {acc}, rejected = {rej}, max err = {np.max(errEst):.2e}")


RKF45 test:
Final y = 0.999592, expected = ~1.0, accepted = 58, rejected = 2, max err = 9.18e-01


In [27]:
# --- Code 3: Library Routine ---

f3 = lambda t, y: -y 
sols3 = solve_ivp(f3, t_span=(0,5), y0=[1.0], method="RK45", rtol=1e-6, atol=1e-9, t_eval=np.linspace(0, 5, 101))

print("Final y = ", sols3.y[0,-1])
print("Expected: ", np.exp(-5))
print("Steps taken: ", len(sols3.t))

Final y =  0.00673795614211217
Expected:  0.006737946999085467
Steps taken:  101
